In [ ]:
#pip install kafka-python
#Install Kafka-python API and also install Apache Kafka on local machine or virtual machine using official documentation
# Link https://kafka-python.readthedocs.io/en/master/
#Follow the documentation to get hands on experience

In [ ]:
from kafka import KafkaConsumer,KafkaProducer
import requests
import json
from requests import Request, Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
from json import dump,loads
import logging
import time

In [ ]:
url = 'https://pro-api.coinmarketcap.com/v1/cryptocurrency/quotes/latest'

parameters={
    "symbol":"BTC",  # Pass the symbol of crypto curreny which you want analyze
    }

headers = {
  'Accepts': 'application/json',
  'X-CMC_PRO_API_KEY': 'your-api-key',  # replace this with your api key obtained from coin market cap developer api
}

In [ ]:
#Create a session
session = Session()
session.headers.update(headers)

In [ ]:
def publish_message(producer_instance,topic_name,key,value):
    
    topic_name="your-topic-name"  # pass your topic name here 
    
    try:
        # I passes 333 because you can call 333 free REST API requests to coin market cap api on a day
        for _ in range(333):   
            #Call the json request by passing url and parameters 
            response = session.get(url, params=parameters)
            
            #read the response recieved from the api-call
            data=response.json()
        
            #key is the timestamp recieved
            key=data["status"]["timestamp"]
            
            #value is the price of BTC recieved
            value=data["data"]["BTC"]["quote"]["USD"]["price"]
            
            #Create a json file by specifying crypto and price . This will be useful if you are analyzing 
            #more than one crypto currency at a time
            
            value1=json.dumps({"DateTime":key,"Price":value})
            
            #Kafka stores data in byte form. So it is irrespective of data you pass
            #You need to serialize when passing the data. There are many forms like Apache Avro, gzip etc.
            #key_bytes=bytes(str(key),encoding="utf-8")
            key_bytes=key.encode(encoding="utf-8")
            
            value_bytes=bytes(value1,encoding="utf-8")
            
            print("RESPONSE_CODE=",response.status_code) #should return 200
            print("CONTENT_TYPE_RECIEVED=",response.headers["Content-Type"]) #application.json
            
            #Now send the data into kafka using the Producer
            producer_instance.send(topic_name,key=key_bytes,value=value_bytes)
            #producer.subscribe can also be used.
            
            #As only 333 request can be send, we need the thread to be waiting.
            time.sleep(20) #wait for "x" sec till next call
        
        #We need to clean the producer after the work is done
        producer_instance.flush()
        
    except Exception as e:
        print("Error writing data into Kafka")
        print(str(e))

In [ ]:
def connect_kafka_producer():
    kafka_producer=None  #Intially not connected to any broker
    try:
        #I'm using local host because I'm running on my local machine.
        #For cases like aws EC2 instance you need to pass the i.p address
        kafka_producer=KafkaProducer(bootstrap_servers=["localhost:9092"])   
        
    except Exception as e:
        print("Error while connecting Producer")
        print(str(e))
        
    finally:
        return kafka_producer

In [ ]:
if __name__=="__main__":
    #Create a producer instance
    producer_instance=connect_kafka_producer()
    
    publish_message(producer_instance,None,None,None)